In [3]:
%pip install tensorflow cv2 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [4]:
#for int 8
import cv2
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="best-int8 (1).tflite")
interpreter.allocate_tensors()

# Get model input details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# Load video
cap = cv2.VideoCapture("mixkit-potholes-in-a-rural-road-25208-hd-ready.mp4")  # ⬅️ Replace with your input video path
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define output video writer
out = cv2.VideoWriter('int8.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Function to preprocess frame for model
def preprocess(frame):
    resized = cv2.resize(frame, (input_shape[2], input_shape[1]))
    input_data = np.expand_dims(resized, axis=0).astype(np.uint8)  # Cast to uint8 for quantized model
    return input_data

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_tensor = preprocess(frame)
    interpreter.set_tensor(input_details[0]['index'], input_tensor) #this is for int8
    interpreter.invoke()

    # NOTE: You must modify the next part depending on your TFLite model's output format
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    pothole_count = 0
    for detection in output_data:
        x_center, y_center, w, h, conf, cls = detection
        if conf > 0.5 and int(cls) == 0:  # Assuming pothole class is 0
            x = int((x_center - w / 2) * width)
            y = int((y_center - h / 2) * height)
            w_box = int(w * width)
            h_box = int(h * height)
            cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)
            pothole_count += 1

    # Draw count
    cv2.putText(frame, f'Potholes: {pothole_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)

    # Optional: Show live result
    cv2.imshow('Pothole Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'

In [ ]:
#for float32 model inference with OpenCV and TensorFlow Lite
import cv2
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="best-fp16 (2).tflite")
interpreter.allocate_tensors()

# Get model input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# Print input and output details for debugging
print("Input details:", input_details)
print("Output details:", output_details)

# Load video
cap = cv2.VideoCapture("mixed.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define output video writer
out = cv2.VideoWriter('output_with_detections.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Function to preprocess frame for model
def preprocess(frame):
    resized = cv2.resize(frame, (input_shape[2], input_shape[1]))
    input_data = np.expand_dims(resized, axis=0).astype(np.float32)
    input_data /= 255.0
    return input_data

# Function to process detections for mobile-friendly output
def process_detections(output_data, width, height, conf_threshold=0.4):
    detections = []
    for det in output_data:
        conf = det[4]
        cls = int(det[5]) if len(det) > 5 else 0
        if conf > conf_threshold:
            x_center, y_center, w, h = det[:4]
            x1 = int((x_center - w / 2) * width)
            y1 = int((y_center - h / 2) * height)
            x2 = int((x_center + w / 2) * width)
            y2 = int((y_center + h / 2) * height)
            detections.append({
                'box': [x1, y1, x2, y2],
                'confidence': float(conf),
                'class': cls
            })
    return detections

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_tensor = preprocess(frame)
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()

    # Get model output
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    # Process detections for mobile integration
    detections = process_detections(output_data, width, height, conf_threshold=0.5)
    print(f"Frame {frame_count}: {detections}")

    # Draw detections on frame
    pothole_count = 0
    for det in detections:
        x1, y1, x2, y2 = det['box']
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        pothole_count += 1

    # Draw count
    cv2.putText(frame, f'Potholes: {pothole_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)

    # Optional: Show live result
    cv2.imshow('Pothole Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()

c:\Users\supre\anaconda3\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
